In [2]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_image_fillter_obj/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_xml_fillter_obj/'

xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'

os.system('mkdir -p ' + generated_xml_dir)
os.system('mkdir -p ' + generated_image_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir
BKG_IMG_DIR = '/data/darknet/python/background/'

# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
bkg_images = os.listdir(BKG_IMG_DIR)
bkg_images.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']

last_frame_bboxes = []
for xml_name in seed_xml_names:
    #print(xml_name)       
    if(not xml_name.endswith('.xml')):
        continue
    
    #-------------------temp code start
    img_data = cv2.imread(BKG_IMG_DIR + '00002100289.jpg')
    #-------------------temp code end
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    this_frame_bboxes = []
    boxes = []
    objs = []
    b = []
    #wrong_detection_objs = [[130, 311, 226, 475]]
    #objs_iter = root.iter('object')
    N = 0
    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        stable_bbox_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        this_frame_bboxes.append(b)
        if(len(last_frame_bboxes)>0):
            for stable_bbox in last_frame_bboxes:
                if(IOU(stable_bbox, b) > 0.85):
                    #print(b)
                    root.remove(obj)
                    stable_bbox_removed = True
                    N = N+1
                    break
            if(stable_bbox_removed):
                continue
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 and bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue
        else:
            for box in boxes:
                if(IOU(box, b) > 0.6):
                    print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
                    root.remove(obj)
                    continue
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))          
        boxes.append(b)
        objs += obj
    last_frame_bboxes = this_frame_bboxes.copy()  
    #print(boxes)
    
    if(len(boxes) == 0):
        continue
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes, img_data)
    tree.write(GENE_XML_DIR + xml_name)

add:28.0:59.0
add:71.0:87.0
add:35.0:91.0
add:61.0:41.0
add:17.0:38.0
add:31.0:51.0
remove:13.0:31.0
add:17.0:41.0
add:18.0:42.0
add:18.0:32.0
add:36.0:62.0
add:17.0:44.0
add:18.0:30.0
add:17.0:42.0
add:31.0:33.0
add:17.0:31.0
add:18.0:39.0
add:30.0:43.0
remove:18.0:39.0
add:18.0:39.0
remove:15.0:40.0
remove:15.0:41.0
add:38.0:73.0
add:17.0:31.0
add:28.0:46.0
add:21.0:41.0
add:31.0:42.0
remove:21.0:41.0
add:21.0:41.0
add:37.0:78.0
remove:16.0:41.0
remove:14.0:35.0
add:20.0:33.0
add:21.0:35.0
remove:16.0:27.0
add:57.0:46.0
add:20.0:44.0
add:21.0:45.0
add:39.0:85.0
remove:16.0:38.0
remove:13.0:35.0
add:65.0:48.0
add:57.0:41.0
add:35.0:48.0
add:17.0:35.0
add:28.0:51.0
add:37.0:52.0
remove:14.0:34.0
add:18.0:33.0
remove:14.0:37.0
remove:14.0:34.0
add:27.0:51.0
add:68.0:48.0
add:20.0:38.0
add:17.0:37.0
add:20.0:39.0
remove:15.0:27.0
add:18.0:167.0
add:42.0:101.0
remove:16.0:40.0
remove:13.0:33.0
add:61.0:51.0
add:29.0:51.0
add:26.0:37.0
remove:29.0:51.0
add:29.0:51.0
add:18.0:31.0
add:34.0:

add:199.0:83.0
add:33.0:62.0
add:24.0:42.0
remove:16.0:45.0
add:26.0:57.0
remove:12.0:48.0
add:23.0:55.0
add:38.0:60.0
remove:12.0:37.0
add:18.0:36.0
add:17.0:44.0
remove:16.0:31.0
add:200.0:84.0
add:28.0:62.0
add:22.0:61.0
remove:16.0:44.0
add:32.0:56.0
add:17.0:28.0
remove:15.0:43.0
add:18.0:30.0
remove:14.0:47.0
remove:14.0:56.0
add:17.0:54.0
remove:10.0:36.0
remove:14.0:34.0
add:21.0:38.0
add:21.0:29.0
remove:11.0:32.0
remove:12.0:50.0
add:178.0:74.0
add:20.0:72.0
add:24.0:67.0
remove:15.0:53.0
add:18.0:41.0
remove:14.0:44.0
remove:10.0:35.0
add:33.0:67.0
add:27.0:56.0
add:21.0:39.0
add:18.0:25.0
add:31.0:45.0
add:80.0:60.0
remove:13.0:45.0
remove:12.0:30.0
add:19.0:67.0
add:25.0:62.0
add:79.0:80.0
add:23.0:59.0
remove:11.0:34.0
add:25.0:58.0
add:26.0:51.0
remove:15.0:49.0
remove:15.0:50.0
remove:12.0:31.0
remove:16.0:29.0
add:39.0:63.0
remove:15.0:31.0
add:18.0:27.0
add:27.0:67.0
add:24.0:56.0
add:26.0:66.0
add:21.0:59.0
add:18.0:50.0
add:28.0:48.0
remove:11.0:35.0
add:29.0:58.0
r

add:20.0:59.0
add:25.0:50.0
add:371.0:311.0
add:25.0:62.0
add:23.0:55.0
add:22.0:66.0
remove:15.0:43.0
add:23.0:34.0
add:24.0:56.0
remove:16.0:43.0
remove:13.0:69.0
remove:12.0:39.0
remove:10.0:35.0
add:22.0:44.0
remove:10.0:33.0
remove:10.0:31.0
add:41.0:74.0
add:17.0:35.0
remove:16.0:47.0
remove:14.0:33.0
remove:14.0:40.0
remove:12.0:37.0
add:19.0:37.0
remove:10.0:31.0
remove:10.0:30.0
add:549.0:336.0
add:226.0:89.0
add:24.0:63.0
add:28.0:76.0
add:24.0:63.0
add:19.0:63.0
add:22.0:70.0
add:23.0:53.0
remove:14.0:47.0
add:19.0:66.0
add:23.0:66.0
add:27.0:57.0
add:22.0:35.0
remove:13.0:34.0
remove:15.0:43.0
remove:12.0:35.0
add:20.0:39.0
add:17.0:45.0
add:21.0:56.0
remove:12.0:43.0
remove:11.0:34.0
remove:10.0:30.0
remove:15.0:28.0
remove:12.0:44.0
add:26.0:94.0
add:23.0:67.0
add:31.0:77.0
add:19.0:70.0
add:26.0:76.0
add:26.0:61.0
add:17.0:51.0
remove:16.0:46.0
remove:14.0:42.0
remove:15.0:37.0
remove:15.0:47.0
remove:13.0:41.0
remove:13.0:68.0
remove:12.0:40.0
add:25.0:56.0
remove:11.0:

add:328.0:142.0
add:246.0:95.0
add:207.0:76.0
add:26.0:62.0
add:19.0:33.0
add:19.0:62.0
add:22.0:37.0
add:17.0:51.0
add:18.0:18.0
add:25.0:50.0
add:21.0:37.0
remove:14.0:23.0
remove:14.0:27.0
add:17.0:21.0
remove:14.0:25.0
add:170.0:81.0
add:21.0:55.0
add:54.0:94.0
add:73.0:67.0
add:20.0:62.0
add:102.0:94.0
add:23.0:65.0
add:22.0:67.0
add:19.0:43.0
add:21.0:65.0
add:23.0:62.0
add:21.0:35.0
add:17.0:54.0
add:18.0:70.0
add:25.0:52.0
remove:13.0:36.0
remove:13.0:36.0
add:17.0:28.0
add:266.0:145.0
add:223.0:83.0
add:26.0:57.0
add:22.0:61.0
add:26.0:58.0
add:24.0:53.0
add:18.0:64.0
add:20.0:46.0
add:21.0:68.0
add:17.0:45.0
add:21.0:56.0
remove:13.0:41.0
remove:16.0:39.0
remove:13.0:37.0
add:19.0:49.0
add:20.0:62.0
add:19.0:37.0
add:22.0:54.0
add:158.0:92.0
add:137.0:122.0
add:27.0:49.0
add:21.0:54.0
add:24.0:64.0
add:21.0:60.0
add:50.0:96.0
add:235.0:126.0
add:23.0:62.0
add:22.0:62.0
add:22.0:59.0
add:25.0:51.0
add:27.0:51.0
remove:14.0:42.0
add:20.0:66.0
remove:15.0:27.0
add:20.0:57.0
remo

add:47.0:51.0
add:34.0:82.0
add:24.0:57.0
add:29.0:62.0
add:32.0:86.0
add:36.0:78.0
add:17.0:51.0
add:44.0:100.0
add:24.0:58.0
add:18.0:44.0
add:24.0:57.0
add:19.0:63.0
add:21.0:41.0
add:17.0:46.0
add:32.0:77.0
add:30.0:108.0
add:30.0:121.0
remove:15.0:33.0
add:27.0:57.0
add:57.0:51.0
add:26.0:53.0
add:21.0:36.0
add:20.0:26.0
remove:16.0:31.0
add:29.0:53.0
add:23.0:62.0
add:29.0:56.0
remove:15.0:30.0
add:27.0:53.0
add:34.0:81.0
add:43.0:109.0
add:38.0:74.0
add:45.0:94.0
add:37.0:62.0
add:25.0:62.0
add:22.0:40.0
add:34.0:77.0
remove:15.0:52.0
add:20.0:56.0
add:18.0:48.0
add:23.0:59.0
add:17.0:58.0
add:21.0:57.0
add:25.0:56.0
add:19.0:45.0
add:21.0:52.0
remove:15.0:58.0
add:18.0:57.0
add:18.0:29.0
add:23.0:52.0
add:19.0:21.0
add:26.0:78.0
add:49.0:107.0
add:52.0:111.0
add:27.0:75.0
add:26.0:58.0
remove:16.0:56.0
add:21.0:37.0
add:24.0:62.0
add:17.0:47.0
add:24.0:62.0
remove:16.0:57.0
add:23.0:52.0
add:19.0:44.0
remove:16.0:45.0
add:27.0:56.0
add:22.0:51.0
add:22.0:31.0
add:19.0:42.0
add:

add:252.0:88.0
remove:16.0:39.0
add:17.0:43.0
remove:15.0:68.0
add:18.0:58.0
add:21.0:63.0
add:18.0:59.0
remove:13.0:69.0
add:23.0:63.0
add:25.0:67.0
add:17.0:37.0
add:72.0:90.0
add:17.0:40.0
add:20.0:58.0
add:20.0:78.0
remove:14.0:62.0
remove:13.0:29.0
remove:14.0:38.0
add:236.0:102.0
add:20.0:64.0
add:18.0:42.0
add:19.0:62.0
add:19.0:56.0
add:21.0:62.0
remove:16.0:35.0
remove:14.0:37.0
remove:13.0:39.0
add:34.0:57.0
add:74.0:86.0
add:79.0:48.0
add:17.0:59.0
add:18.0:53.0
add:40.0:58.0
add:20.0:44.0
add:228.0:89.0
add:24.0:65.0
add:22.0:59.0
remove:15.0:41.0
add:57.0:38.0
add:37.0:48.0
add:20.0:64.0
add:17.0:38.0
add:64.0:58.0
add:32.0:63.0
remove:12.0:32.0
add:17.0:36.0
add:20.0:65.0
remove:15.0:39.0
add:17.0:50.0
add:18.0:57.0
add:288.0:126.0
remove:16.0:45.0
add:51.0:53.0
add:26.0:54.0
add:31.0:48.0
add:224.0:201.0
add:18.0:65.0
remove:16.0:33.0
add:21.0:65.0
add:51.0:42.0
add:35.0:49.0
remove:13.0:97.0
add:19.0:29.0
add:550.0:253.0
add:303.0:107.0
remove:15.0:43.0
remove:14.0:31.0

add:273.0:94.0
add:23.0:59.0
add:18.0:43.0
add:25.0:59.0
add:17.0:42.0
add:21.0:56.0
add:18.0:43.0
remove:11.0:37.0
add:19.0:48.0
add:65.0:145.0
add:36.0:50.0
add:20.0:33.0
remove:14.0:27.0
remove:14.0:22.0
remove:12.0:37.0
add:31.0:32.0
add:146.0:84.0
add:202.0:159.0
add:23.0:58.0
add:20.0:54.0
add:23.0:56.0
add:21.0:64.0
add:17.0:44.0
remove:13.0:38.0
remove:15.0:35.0
add:20.0:40.0
remove:13.0:36.0
add:30.0:49.0
add:25.0:50.0
remove:16.0:43.0
add:369.0:191.0
add:28.0:59.0
add:20.0:56.0
remove:16.0:54.0
remove:13.0:36.0
add:22.0:56.0
remove:16.0:37.0
remove:12.0:34.0
add:18.0:30.0
remove:16.0:32.0
remove:15.0:29.0
remove:16.0:23.0
remove:14.0:28.0
add:506.0:197.0
remove:12.0:27.0
remove:14.0:30.0
remove:12.0:29.0
remove:16.0:29.0
remove:16.0:27.0
add:18.0:25.0
add:17.0:31.0
add:384.0:172.0
remove:13.0:39.0
remove:16.0:34.0
remove:16.0:39.0
add:17.0:34.0
remove:16.0:32.0
remove:10.0:31.0
add:17.0:36.0
remove:11.0:33.0
remove:19.0:14.0
add:26.0:63.0
add:179.0:151.0
add:17.0:48.0
add:19.

add:279.0:88.0
add:534.0:153.0
add:102.0:86.0
remove:14.0:32.0
remove:14.0:31.0
remove:14.0:28.0
add:18.0:23.0
add:263.0:95.0
add:460.0:163.0
remove:13.0:34.0
remove:15.0:27.0
add:30.0:94.0
add:64.0:102.0
add:260.0:95.0
remove:16.0:40.0
add:21.0:93.0
remove:12.0:40.0
add:43.0:73.0
add:247.0:89.0
add:337.0:134.0
add:34.0:105.0
add:21.0:115.0
remove:12.0:40.0
add:237.0:92.0
add:247.0:139.0
add:41.0:109.0
remove:13.0:41.0
remove:13.0:31.0
remove:11.0:35.0
add:19.0:30.0
add:234.0:96.0
add:122.0:149.0
add:22.0:82.0
add:26.0:108.0
remove:16.0:35.0
add:136.0:86.0
add:35.0:98.0
remove:15.0:37.0
add:329.0:177.0
remove:13.0:39.0
add:50.0:112.0
remove:14.0:44.0
remove:15.0:38.0
add:372.0:176.0
add:307.0:174.0
add:43.0:106.0
add:499.0:186.0
remove:15.0:35.0
add:18.0:20.0
add:290.0:165.0
remove:290.0:165.0
add:290.0:165.0
add:475.0:166.0
add:36.0:97.0
add:124.0:102.0
remove:16.0:33.0
add:55.0:132.0
add:351.0:141.0
add:49.0:109.0
add:269.0:94.0
remove:17.0:16.0
remove:12.0:33.0
add:21.0:102.0
add:19

add:158.0:62.0
add:19.0:24.0
add:161.0:147.0
add:23.0:33.0
add:236.0:97.0
add:23.0:32.0
add:118.0:107.0
add:154.0:97.0
add:25.0:34.0
add:37.0:61.0
add:22.0:42.0
add:70.0:51.0
add:23.0:63.0
add:23.0:32.0
add:55.0:46.0
add:38.0:53.0
add:25.0:53.0
add:23.0:63.0
add:18.0:29.0
add:22.0:33.0
add:39.0:63.0
add:25.0:39.0
add:77.0:47.0
add:32.0:51.0
add:147.0:99.0
add:22.0:40.0
add:36.0:55.0
add:23.0:51.0
add:59.0:55.0
add:244.0:93.0
add:238.0:83.0
add:22.0:41.0
add:24.0:35.0
add:31.0:58.0
add:221.0:78.0
add:22.0:43.0
add:170.0:48.0
add:20.0:20.0
add:23.0:54.0
add:21.0:40.0
add:159.0:63.0
add:305.0:97.0
add:26.0:45.0
add:22.0:33.0
add:24.0:38.0
add:31.0:46.0
add:105.0:91.0
add:20.0:51.0
add:19.0:34.0
add:30.0:47.0
add:26.0:55.0
add:275.0:87.0
add:21.0:42.0
remove:15.0:53.0
add:25.0:59.0
add:49.0:53.0
add:23.0:65.0
add:17.0:49.0
add:22.0:55.0
add:26.0:57.0
add:72.0:45.0
add:37.0:63.0
add:31.0:52.0
remove:15.0:53.0
add:24.0:65.0
add:17.0:55.0
add:23.0:60.0
add:31.0:59.0
add:39.0:66.0
remove:15.0: